### Description

- This project is part of Machine Learning course (CSCI 5622) at University of Colorado Boulder
- In this assignment, I developed a regression model that predicts the number of bikes shared from a Seoul Bike Sharing program.
- The data used is subset of `Seoul Bike Sharing Demand Data Set` in UCI repository sampled by instructor.
- It comprised of 13 features and ranges from `1/12/2017` to `30/11/2018`.
- The final metric used was `r-squared` score.
- I recieved `0.93150` for a public and `0.94145` for a private leaderboard using XGBoost model after fine-tuning the parameters.
- Website: https://www.kaggle.com/competitions/csci-5622-ps4-22-fall/overview

### Importing Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import datetime

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

### Building a prediction model

#### Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import datetime
warnings.filterwarnings('ignore')

#### Reading from csv files

In [4]:
XTrain = pd.read_csv('data/XTrain.csv').rename(columns={'Temperature(�C)': 'T', 'Humidity(%)': 'H'
                                                  , 'Wind speed (m/s)':'WS', 'Visibility (10m)':'V'
                                                  , 'Dew point temperature(�C)': 'DP', 'Solar Radiation (MJ/m2)': 'SR'
                                                  , 'Rainfall(mm)': 'R', 'Snowfall (cm)': 'S'
                                                  , 'Functioning Day': 'FD'})
yTrain = pd.read_csv('data/yTrain.csv')
XTest = pd.read_csv('data/XTest.csv').rename(columns={'Temperature(�C)': 'T', 'Humidity(%)': 'H'
                                                  , 'Wind speed (m/s)':'WS', 'Visibility (10m)':'V'
                                                  , 'Dew point temperature(�C)': 'DP', 'Solar Radiation (MJ/m2)': 'SR'
                                                  , 'Rainfall(mm)': 'R', 'Snowfall (cm)': 'S'
                                                  , 'Functioning Day': 'FD'})

#### Preprocessing Data - Rounding/Clipping/Binning data according to the EDA result

In [5]:
# Handling Date column by chaning it from string to date type
# Used apply/lambda function to handle various date format 
XTrain['Date'] = XTrain['Date'].apply(lambda x: datetime.datetime(int(x.split('/')[2]), int(x.split('/')[1]), int(x.split('/')[0])))
XTest['Date'] = XTest['Date'].apply(lambda x: datetime.datetime(int(x.split('/')[2]), int(x.split('/')[1]), int(x.split('/')[0])))

# Removing unnecessary column
Train = XTrain.loc[:, XTrain.columns!='Index']
Test = XTest.loc[:, XTest.columns!='Index']

# Segmenting Date type column into various granuality
Train['Day'] = pd.DatetimeIndex(Train['Date']).day
Train['Month'] = pd.DatetimeIndex(Train['Date']).month
Test['Day'] = pd.DatetimeIndex(Test['Date']).day
Test['Month'] = pd.DatetimeIndex(Test['Date']).month
Train['Weekday'] = pd.DatetimeIndex(Train['Date']).dayofweek
Test['Weekday'] = pd.DatetimeIndex(Test['Date']).dayofweek

# Removing unnecessary column
Train = Train.loc[:, Train.columns!='Date']
Test = Test.loc[:, Test.columns!='Date']

# Rounding column 'T' (Temperature) according to the EDA result
Train['T'] = round(Train['T'])
Test['T'] = round(Test['T'])

# Rounding column 'H' (Humidity) according to the result
Train['H'] = np.clip(round(Train['H']), a_min=0, a_max=100)
Test['H'] = np.clip(round(Test['H']), a_min=0, a_max=100)

# Rounding and clipping column 'WS' (Wind speed) according to the EDA result 
Train['WS'] = np.clip(round(Train['WS']), a_min=0, a_max=6)
Test['WS'] = np.clip(round(Test['WS']), a_min=0, a_max=6)

# Rouding column 'V' (Visibility) according to the EDA result
Train['V'] = round(Train['V'], -2)
Test['V'] = round(Test['V'], -2)

# Rounding column 'DP' (Dew point temperature) according to the EDA result
Train['DP'] = round(Train['DP'])
Test['DP'] = round(Test['DP'])

# Rounding and clipping column 'SR' (Solar Radiation) according to the EDA result
Train['SR'] = np.clip(round(Train['SR']), a_min=0, a_max=4)
Test['SR'] = np.clip(round(Test['SR']), a_min=0, a_max=4)

# Rounding and binning column 'R' (Rainfall) according to the EDA result
Train['R'] = (Train['R']==0).astype('int')
Test['R'] = (Test['R']==0).astype('int')

# Rounding column 'S' (Snowfall) according to the EDA result
Train['S'] = round(Train['S'])
Test['S'] = round(Test['S'])

# Getting rid of samples with 'FD' (Functional Day) eqauls 'False'
# It is based on the obervation that every sample with 'FD'=='False' has zero rented bike
# Samples with 'FD'=='False' will be treated seperately by applying a manual rule (i.e. prediction will be zero when 'FD'=='False')
Train = Train[Train['FD']=='Yes']

#### Preprocessing data - Encoding the original values with average number of rented bikes
- Instead of using one-hot encoding, I encoded the categorical features using the average number of rented bikes to make the prediction more accurate

In [9]:
# Create a list of categorical features
categorical_features = ['Month', 'Hour', 'Seasons', 'Weekday', 'Day', 'Holiday', 'R']

# Temporary dataframe for calculating the average number of rented bikes according to various categorical features
combined = pd.merge(Train, yTrain, left_index=True, right_index=True)

# Calculate the average rented bike count for each categorical feature and store it in a dictionary
avg_dict = {}
for feature in categorical_features:
    avg_dict[feature] = combined.groupby(feature)['Rented Bike Count'].mean()

# Use map to assign the calculated average values to each feature in the Train and Test sets
for feature in categorical_features:
    avg_name = feature + '_avg'  # Create a new column name (e.g., 'Month_avg')
    
    # Map the average values from avg_dict to the corresponding categorical columns
    Train[avg_name] = Train[feature].map(avg_dict[feature])
    Test[avg_name] = Test[feature].map(avg_dict[feature])

# Removing unnecessary columns from both Train and Test
drop_columns = ['FD', 'Day', 'DP', 'Month', 'Hour', 'Seasons', 'Weekday', 'Holiday', 'R']
X = Train.drop(columns=drop_columns)
X_test = Test.drop(columns=drop_columns)

#### Preprocessing - Standardization

In [10]:
# Applying standardization using Scikit-learn's StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

# Getting rid of samples with 'FD' (Functional Day) eqauls 'False'
y = yTrain[XTrain['FD']=='Yes']['Rented Bike Count']

#### Hyperparameter tuning

In [11]:
# I tried various types of machine learning algorithms such as XGBoost, RandomForest, Decision Tree, Polynomial Regression,
# AdaBoost, Deep Neural Networks, and ElasticNet and concluded that XGBoost shows the best performance.
# Fine-tuing XGBRegressor using GridSearchCV
# Carried out several steps to decrease the search spaces
parameters = {'eta': [0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24], 'max_depth': [7]
              , 'reg_lambda': [1.6, 1.7, 1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4]
              , 'colsample_bytree': [0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84]}
model = XGBRegressor()
clf = GridSearchCV(model, parameters, scoring='r2', verbose=3)
clf.fit(X_scaled, y)

print(clf.best_score_)
print(clf.best_params_)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV 1/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.6;, score=0.913 total time=   0.3s
[CV 2/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.6;, score=0.923 total time=   0.1s
[CV 3/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.6;, score=0.918 total time=   0.2s
[CV 4/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.6;, score=0.915 total time=   0.1s
[CV 5/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.6;, score=0.917 total time=   0.2s
[CV 1/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.7;, score=0.912 total time=   0.2s
[CV 2/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.7;, score=0.915 total time=   0.2s
[CV 3/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.7;, score=0.918 total time=   0.1s
[CV 4/5] END colsample_bytree=0.76, eta=0.16, max_depth=7, reg_lambda=1.7;, scor

#### Model training & prediction

In [14]:
# Fitting the model with the best parameters
model = XGBRegressor(**clf.best_params_)
model.fit(X_scaled, y)

# Make predictions on the test set
# When 'FD'=='No', set the prediction values to zero
# When prediction values are below zero, which is not feasible, set the prediction values to zero
# This can be further improved by ensemble method
Test['prediction'] = model.predict(X_test_scaled)
Test.loc[Test['FD']!='Yes', 'prediction'] = 0
Test.loc[Test['prediction']<0, 'prediction'] = 0

# Making an output csv file
Test['prediction'].reset_index().rename(columns={'index': 'Index', 'prediction':'Rented Bike Count'}).to_csv('result/fin_prediction.csv', index=False)